In [ ]:
#linkedin 
#find the jobs "Data Analyst" in "California" with type "full-time"

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import requests
import re
import time
import json
import pymongo

In [ ]:
# driver = webdriver.Chrome(executable_path = '/Users/oscarhui/Desktop/422/HW4/chromedriver')
# driver.implicitly_wait(5)
# driver.set_script_timeout(5)
# driver.set_page_load_timeout(5)

In [122]:
start = 1
num_of_pages = 8

for i in range(num_of_pages):
    URL = f"https://www.linkedin.com/jobs/search/?currentJobId=3502778692&distance=200.0&geoId=102095887&keywords=Data%20Analyst&start={start}"
    start += 25
    Session_requests = requests.Session()
    user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56'} 

    page = Session_requests.get(URL,headers = user_agent)
    # print(page.content)
    cookies = Session_requests.cookies.get_dict()
    # print(cookies)

    full_page = BeautifulSoup(page.content,'html.parser')
    file = open(f'linkedin_data_analyst_position{i+1}.htm',"w")
    # #Write the content into the html file
    file.write(str(full_page))
    time.sleep(3)

    # #Close the file
    # file.close()



In [123]:
for i in range(num_of_pages):
    with open('linkedin_data_analyst_position{}.htm'.format(i+1),'r', encoding='utf-8') as f:
        page = f.read()

    soup = BeautifulSoup(page,'html.parser')
    jobs = soup.find_all('a', class_ = "base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]")

    job_link_list = []
    j = 25*i 
    for job in jobs:
        job_link = job['href']
        job_link_list.append(job_link)
        job_page = requests.get(job_link, headers=user_agent,cookies = cookies)
        job_web = BeautifulSoup(job_page.content, "html.parser")
        with open(f'job{(j+1):03d}.htm','w')as f:
            f.write(str(job_web))
        time.sleep(3)
                
        j+=1



from each job page get the info  
1.position name (position_name)
2.company name (company_name)
3.location (location)
4.post date (post_date)
5.about us (about_position)

In [ ]:
position_name = []
company_name = []
location = []
post_date = []
required_skils = []



for j in range(0,200):
    with open('job{:03d}.htm'.format(j+1),'r', encoding='utf-8') as f:
        job_detail_page = f.read()
    
    position_details_soup = BeautifulSoup(job_detail_page,'html.parser')

    p_name = position_details_soup.find_all('h1', class_ = "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title")
    if p_name:
        p_name = p_name[0].text.strip()
    else:
        p_name = "N/A"  
    position_name.append(p_name)
    
    c_name = position_details_soup.find_all('span', class_ = "topcard__flavor")
    if c_name:
        c_name = c_name[0].text.strip()
    else:
        c_name = "N/A"  
    company_name.append(c_name)

    loca = position_details_soup.find('span', class_ = "topcard__flavor topcard__flavor--bullet")
    if loca:
        loca = loca.text.strip()
    else:
        loca = "N/A"
    location.append(loca)
    postdate = position_details_soup.find('span', class_ = "posted-time-ago__text topcard__flavor--metadata")
    if postdate:
        postdate = postdate.text.strip()
    else:
        postdate = "N/A"
    post_date.append(postdate)
    about = position_details_soup.find('div', class_ = "show-more-less-html__markup").text
    required_skils.append(about)
print(position_name)
print(company_name)
print(location)
print(post_date)
# print(required_skils)

### company information
#### column name ---- stored list


company links --- company_linklist      
about us --- aboutus_list
company external website link --- web_linklist      
number of employees on linkedin --- emp_onlinkedin_numlist     
headquarters --- headquarters_list
organization type --- org_type_list
founded year list --- founded_yr_list
specialties --- specialty_list
primary location --- main_locationlist

In [56]:
# store all the companies' links
company_linklist = []

for j in range(0,200):
    with open('job{:03d}.htm'.format(j+1),'r', encoding='utf-8') as f:
        job_detail_page = f.read()
    
    position_details_soup = BeautifulSoup(job_detail_page,'html.parser')
    company = position_details_soup.find('a', class_ = "topcard__org-name-link topcard__flavor--black-link")
    # for company in companies:
    # print(company)
    company_link = company['href']
    #     company_linklist.append(company['href'])
#     company_link = company.get('href')
    company_linklist.append(company_link)

# print(company_linklist)
print(len(company_linklist))
# # company_linklist

    

In [ ]:
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56'} 

for j in range(0,200):
    company_page = requests.get(company_linklist[j], headers=user_agent)
    # company_web = BeautifulSoup(job_page.content, "html.parser")
    # print(company_page.content)
    company_web = BeautifulSoup(company_page.content, "html.parser")
    print(company_web)
    with open(f'job{(j+1):03d}.htm','w')as f:
            f.write(str(company_web))
    # with open(f'company{(j+1):03d}.htm','wb')as f:
    #         f.write(company_page.content)


In [137]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### scrape the page of companies

for i in range(0,200):
# Set up the Chrome driver
        # driver = webdriver.Chrome()
        chrome_options = webdriver.ChromeOptions()
        user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
        chrome_options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(options=chrome_options)
# Navigate to the URL
        url = company_linklist[i]
        driver.get(url) 

        # Click on the sign-in button
        wait = WebDriverWait(driver, 5)
        sign_in_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'contextual-sign-in-modal__modal-dismiss-icon.lazy-loaded')))
        # sign_in_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'full-lockout__action.grow.m-2.max-w-lg.btn-md.btn-primary')))

        # sign_in_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'sign-in-modal')))
# full-lockout__action grow m-2 max-w-lg btn-md btn-primary
        sign_in_button.click()
        time.sleep(15)

        # # Enter the email address
        # email_field = wait.until(EC.presence_of_element_located((By.ID, 'organization_guest_contextual-sign-in_sign-in-modal_session_key')))
        # email_field.send_keys('oscarhui9525@gmail.com')
        # email_field.send_keys(Keys.RETURN)
        # time.sleep(5)

        # # Enter the password
        # password_field = wait.until(EC.presence_of_element_located((By.ID, 'organization_guest_contextual-sign-in_sign-in-modal_session_password')))
        # password_field.send_keys('JornJimDavid4x2')
        # password_field.send_keys(Keys.RETURN)
        
        # time.sleep(50)
        # # sign_in_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'sign-in-modal')))
        # # sign_in_button.click()
        # # Write the page source to a file
        # # element_present = EC.presence_of_element_located((By.ID, 'ember27'))
        # # wait.until(element_present)

        with open(f'company_page/company{(i+1):03d}.htm', 'w') as f:
            f.write(driver.page_source)
            i =i+1
        time.sleep(5)
        # driver.quit()

        # driver.quit()
# time.sleep(10)
# # Close the browser
# driver.quit()
 

In [234]:
aboutus_list = []
web_linklist = []
emp_onlinkedin_numlist = []
industry_list = []
company_sizelist = []
headquarters_list = []
org_type_list = []
founded_yr_list =[]
specialty_list = []
main_locationlist = []

for j in range(0,200):
    if j == 99: # There is no info for company 100 so there is no file downloaded
       aboutus_list.append('None')
       web_linklist.append('None')
       industry_list.append('None')
       company_sizelist.append('None')
       headquarters_list.append('None')
       emp_onlinkedin_numlist.append('Not mentioned in Linkedin')
       main_locationlist.append('None')
       specialty_list.append('None')
       founded_yr_list.append('None')
       org_type_list.append('None')


       continue

    with open('company_page/company{:03d}.htm'.format(j+1),'r', encoding='utf-8') as f:
        company_detail_page = f.read()
    company_details_soup = BeautifulSoup(company_detail_page,'html.parser')
    # print(company_details_soup)

    # about us
    about_us = company_details_soup.find('p', {'data-test-id': 'about-us__description'})
    if about_us:
        about_us = company_details_soup.find('p', {'data-test-id': 'about-us__description'}).text.strip()
        aboutus_list.append(about_us)
    else: 
        aboutus_list.append('None')

    # company website link
    web_link = company_details_soup.find('a', {'data-tracking-control-name':'about_website'})
    if web_link:
        web_link = company_details_soup.find('a', {'data-tracking-control-name':'about_website'})['href']
        web_linklist.append(web_link)
    else:
        web_linklist.append('None')
    # print(web_link)

    # industry
    industry = company_details_soup.find('div', {'data-test-id': 'about-us__industries'})
    # strip and replace 'Industries' and \n from results
    if industry:
        industry = company_details_soup.find('div', {'data-test-id': 'about-us__industries'}).text.strip()
        industry=' '.join(industry.split()[-5:]).strip()  
        industry = industry.replace('Industries ','')
        industry_list.append(industry)
    else:
        industry_list.append('None')
  
    # company size
    company_size = company_details_soup.find('div', {'data-test-id': 'about-us__size'})
    if company_size:
        company_size = company_details_soup.find('div', {'data-test-id': 'about-us__size'}).text.strip()
        company_size=' '.join(company_size.split()[-2:]).strip()  
        company_size = company_size.replace('Company size ','')
        company_sizelist.append(company_size)
    else:
        company_sizelist.append('None')
    # print(company_size)

    # headquarters
    headquarters = company_details_soup.find('div', {'data-test-id': 'about-us__headquarters'})
    if headquarters:
        headquarters = company_details_soup.find('div', {'data-test-id': 'about-us__headquarters'}).text.strip()
        headquarters=' '.join(headquarters.split()[-4:]).strip()  
        headquarters = headquarters.replace('Headquarters ','')
        headquarters_list.append(headquarters)
    else:
        headquarters_list.append('None')
  

    # organization type
    org_type = company_details_soup.find('div', {'data-test-id': 'about-us__organizationType'})
    if org_type:
        org_type = company_details_soup.find('div', {'data-test-id': 'about-us__organizationType'}).text.strip()
        org_type=' '.join(org_type.split()[-4:]).strip()  
        org_type = org_type.replace('Type ','')
        org_type_list.append(org_type)
    else:
        org_type_list.append('None')


    # founded year
    founded_yr = company_details_soup.find('div', {'data-test-id': 'about-us__foundedOn'})
    if founded_yr:
        founded_yr = company_details_soup.find('div', {'data-test-id': 'about-us__foundedOn'}).text.strip()
        founded_yr=' '.join(founded_yr.split()[-4:]).strip()  
        founded_yr = founded_yr.replace('Founded ','')
        founded_yr_list.append(founded_yr)
    else:
        founded_yr_list.append('None')

    # specialty
    specialty = company_details_soup.find('div', {'data-test-id': 'about-us__specialties'})
    if specialty:
        specialty = company_details_soup.find('div', {'data-test-id': 'about-us__specialties'}).text.strip()
        specialty=' '.join(specialty.split()[-100:]).strip()  
        specialty = specialty.replace('Specialties ','')
        specialty_list.append(specialty)
    else:
        specialty_list.append('None')
    
    # primary location
    main_location = company_details_soup.find('div', id = "address-0")
    if main_location:
        main_location = company_details_soup.find('div', id = "address-0").text.strip()
        main_location=' '.join(main_location.split()[-100:]).strip()  
        main_locationlist.append(main_location)
    else:
        main_locationlist.append('None')
    
    # number of employees on linkedin
    emp_num_info = company_details_soup.find('a', class_ = "face-pile__cta self-center link-no-visited-state")
    try: 
        emp_num_info = company_details_soup.find('a', class_ = "face-pile__cta self-center link-no-visited-state").text.strip() 
        pattern = r'\d{1,3}(?:,\d{3})*'  # regular expression to match numbers with commas ?: for non capturing group

        result = re.findall(pattern, emp_num_info)
        if result:
            emp_num = result[0].replace(',', '')  # remove commas from the matched number
            emp_onlinkedin_numlist.append(emp_num)
        else:
            emp_onlinkedin_numlist.append('Not mentioned in Linkedin')
    except:
            emp_onlinkedin_numlist.append('Not mentioned in Linkedin')
    # print(emp_num)
    j += j


### put into MongoDB

In [231]:
import os
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [263]:
client = MongoClient('mongodb://localhost:27017/')



# Create a new database and collection
# Initialize the 'rarity' dictionary with None value
db = client['DDR_final_project']
collection = db['Linkedin_job_info']
collection.delete_many({}) # make sure every time when I run this function, the collection is NA


for j in range(len(position_name)):
        job_data = {
        'position_name': None, 
        'job_location': None,
        'job_post_data': None,
        'job_required_skills': None,
        'company_info': []
    }
        job_data['position_name'] = position_name[j]
        job_data['job_location'] = location[j]
        job_data['job_post_data'] = post_date[j]
        job_data['job_required_skills'] = required_skils[j]
        job_data['company_info'] = []
        company_data = {
            'company_name': company_name[j],
            'company_description': aboutus_list[j],
            'company_external_link': web_linklist[j],
            'number_of_employees_on_linkedin': emp_onlinkedin_numlist[j],
            'industry': industry_list[j],
            'company_size': company_sizelist[j],
            'headquarters': headquarters_list[j],
            'organization_type': org_type_list[j],
            'founded_year': founded_yr_list[j],
            'specialties': specialty_list[j],
            'main_location': main_locationlist[j]
        }
        job_data['company_info'].append(company_data)
        j=j+1

        collection.insert_one(job_data)
        # print(list(collection.find()))